<a href="https://colab.research.google.com/github/almunawar12/Artificial-Intelligence/blob/main/Tugas_05_M_Rifki_Almunawar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Goal Stack Planning**

In [ ]:
# Base Classes
# PREDICATE - ON, ONTABLE, CLEAR, HOLDING, ARMEMPTY
class PREDICATE:
  def __str__(self):
    pass
  def __repr__(self):
    pass
  def __eq__(self, other):
    pass
  def __hash__(self):
    pass
  def get_action(self, world_state):
    pass

# OPERATIONS - Stack, Unstack, Pickup, Putdown
class Operation:
  def __str__(self):
    pass
  def __repr__(self):
    pass
  def __eq__(self, other):
    pass
  def precondition(self):
    pass
  def delete(self):
    pass
  def add(self):
    pass
    
class ON(PREDICATE):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
  def __str__(self):
    return "ON({X},{Y})".format(X=self.X, Y=self.Y)
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def __hash__(self):
    return hash(str(self))
  def get_action(self, world_state):
    return StackOp(self.X, self.Y)

class ONTABLE(PREDICATE):
  def __init__(self, X):
    self.X = X
  def __str__(self):
    return "ONTABLE({X})".format(X=self.X)
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def __hash__(self):
    return hash(str(self))
  def get_action(self, world_state):
    return PutdownOp(self.X)

class CLEAR(PREDICATE):
  def __init__(self, X):
    self.X = X
  def __str__(self):
    return "CLEAR({X})".format(X=self.X)
    self.X = X
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def __hash__(self):
    return hash(str(self))
  def get_action(self, world_state):
    for predicate in world_state:
      # If Block is on another block, unstack
      if isinstance(predicate, ON) and predicate.Y == self.X:
        return UnstackOp(predicate.X, predicate.Y)
    return None

class HOLDING(PREDICATE):
  def __init__(self, X):
    self.X = X
  def __str__(self):
    return "HOLDING({X})".format(X=self.X)
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def __hash__(self):
    return hash(str(self))
  def get_action(self, world_state):
    X = self.X
    # If block is on table, pick up
    if ONTABLE(X) in world_state:
      return PickupOp(X)
  # If block is on another block, unstack
    else:
      for predicate in world_state:
        if isinstance(predicate, ON) and predicate.X == X:
          return UnstackOp(X, predicate.Y)

class ARMEMPTY(PREDICATE):
  def __init__(self):
    pass
  def __str__(self):
    return "ARMEMPTY"
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def __hash__(self):
    return hash(str(self))
  def get_action(self, world_state=[]):
    for predicate in world_state:
      if isinstance(predicate, HOLDING):
        return PutdownOp(predicate.X)
    return None

class StackOp(Operation):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
  def __str__(self):
    return "STACK({X},{Y})".format(X=self.X, Y=self.Y)
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def precondition(self):
    return [CLEAR(self.Y), HOLDING(self.X)]
  def delete(self):
    return [CLEAR(self.Y), HOLDING(self.X)]
  def add(self):
    return [ARMEMPTY(), ON(self.X, self.Y)]

class UnstackOp(Operation):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
  def __str__(self):
    return "UNSTACK({X},{Y})".format(X=self.X, Y=self.Y)
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def precondition(self):
    return [ARMEMPTY(), ON(self.X, self.Y), CLEAR(self.X)]
  def delete(self):
    return [ARMEMPTY(), ON(self.X, self.Y)]
  def add(self):
    return [CLEAR(self.Y), HOLDING(self.X)]

class PickupOp(Operation):
  def __init__(self, X):
    self.X = X
  def __str__(self):
    return "PICKUP({X})".format(X=self.X)
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def precondition(self):
    return [CLEAR(self.X), ONTABLE(self.X), ARMEMPTY()]
  def delete(self):
    return [ARMEMPTY(), ONTABLE(self.X)]
  def add(self):
    return [HOLDING(self.X)]

class PutdownOp(Operation):
  def __init__(self, X):
    self.X = X
  def __str__(self):
    return "PUTDOWN({X})".format(X=self.X)
  def __repr__(self):
    return self.__str__()
  def __eq__(self, other):
    return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
  def precondition(self):
    return [HOLDING(self.X)]
  def delete(self):
    return [HOLDING(self.X)]
  def add(self):
    return [ARMEMPTY(), ONTABLE(self.X)]
  def isPredicate(obj):
    predicates = [ON, ONTABLE, CLEAR, HOLDING, ARMEMPTY]
    for predicate in predicates:
      if isinstance(obj, predicate):
        return True
    return False

def isOperation(obj):
  operations = [StackOp, UnstackOp, PickupOp, PutdownOp]
  for operation in operations:
    if isinstance(obj, operation):
      return True
  return False

def arm_status(world_state):
  for predicate in world_state:
    if isinstance(predicate, HOLDING):
      return predicate
  return ARMEMPTY()

class GoalStackPlanner:
  def __init__(self, initial_state, goal_state):
    self.initial_state = initial_state
    self.goal_state = goal_state
  def get_steps(self):
    # Store Steps
    steps = []
    # Program Stack
    stack = []
    # World State/Knowledge Base
    world_state = self.initial_state.copy()
    # Initially push the goal_state as compound goal onto the stack
    stack.append(self.goal_state.copy())
    # Repeat until the stack is empty
    while len(stack) != 0:
    # Get the top of the stack
      stack_top = stack[-1]
    # If Stack Top is Compound Goal, push its unsatisfied goals onto stack
      if type(stack_top) is list:
        compound_goal = stack.pop()
        for goal in compound_goal:
          if goal not in world_state:
            stack.append(goal)
      # If Stack Top is an action
      elif isOperation(stack_top):
      # Peek the operation
        operation = stack[-1]
        all_preconditions_satisfied = True
      # Check if any precondition is unsatisfied and push it onto program stack
        for predicate in operation.delete():
          if predicate not in world_state:
            all_preconditions_satisfied = False
            stack.append(predicate)
      # If all preconditions are satisfied, pop operation from stack and execute it
        if all_preconditions_satisfied:
          stack.pop()
          steps.append(operation)
          for predicate in operation.delete():
            world_state.remove(predicate)
          for predicate in operation.add():
            world_state.append(predicate)
      # If Stack Top is a single satisfied goal
      elif stack_top in world_state:
        stack.pop()
      # If Stack Top is a single unsatisfied goal
      else:
        unsatisfied_goal = stack.pop()
      # Replace Unsatisfied Goal with an action that can complete it
        action = unsatisfied_goal.get_action(world_state)
        stack.append(action)
      # Push Precondition on the stack
        for predicate in action.precondition():
          if predicate not in world_state:
            stack.append(predicate)
    return steps

if __name__ == '__main__':
  initial_state = [
      ON('B', 'A'), ONTABLE('A'), ONTABLE('C'), ONTABLE('D'), CLEAR('B'), CLEAR('C'), CLEAR('D'), ARMEMPTY()
  ]
  goal_state = [
      ON('B', 'D'), ON('C', 'A'),
      ONTABLE('D'), ONTABLE('A'),
      CLEAR('B'), CLEAR('C'),
      ARMEMPTY()
  ]
  goal_stack = GoalStackPlanner(initial_state=initial_state,
  goal_state=goal_state)
  steps = goal_stack.get_steps()
  print(steps)

[PICKUP(C), PUTDOWN(C), UNSTACK(B,A), PUTDOWN(B), PICKUP(C), STACK(C,A), PICKUP(B), STACK(B,D)]


# **Tic-Tac-Toe**

In [ ]:
def draw_board(board):
    """
     Fungsi untuk menggambar papan permainan
    """
    print(board[0] + "|" + board[1] + "|" + board[2])
    print("-+-+-")
    print(board[3] + "|" + board[4] + "|" + board[5])
    print("-+-+-")
    print(board[6] + "|" + board[7] + "|" + board[8])
    
def check_win(board, player):
    """
    Fungsi untuk memeriksa apakah seorang pemain menang
    """
    win_positions = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
    for positions in win_positions:
        if board[positions[0]] == player and board[positions[1]] == player and board[positions[2]] == player:
            return True
    return False
    
def tic_tac_toe():
    """
    Fungsi untuk menjalankan permainan Tic Tac Toe
    """
    board = [" ", " ", " ", " ", " ", " ", " ", " ", " "]
    players = ["X", "O"]
    turn = 0
    while True:
        draw_board(board)
        print("Player " + players[turn] + "'s turn.")
        position = int(input("Choose a position (1-9): ")) - 1
        if board[position] != " ":
            print("Position already taken. Choose another one.")
            continue
        board[position] = players[turn]
        if check_win(board, players[turn]):
            draw_board(board)
            print("Player " + players[turn] + " wins!")
            break
        if " " not in board:
            draw_board(board)
            print("It's a tie!")
            break
        turn = (turn + 1) % 2

if __name__ == "__main__":
    tic_tac_toe()


 | | 
-+-+-
 | | 
-+-+-
 | | 
Player X's turn.
Choose a position (1-9): 1
X| | 
-+-+-
 | | 
-+-+-
 | | 
Player O's turn.
Choose a position (1-9): 3
X| |O
-+-+-
 | | 
-+-+-
 | | 
Player X's turn.
Choose a position (1-9): 4
X| |O
-+-+-
X| | 
-+-+-
 | | 
Player O's turn.
Choose a position (1-9): 6
X| |O
-+-+-
X| |O
-+-+-
 | | 
Player X's turn.
Choose a position (1-9): 7
X| |O
-+-+-
X| |O
-+-+-
X| | 
Player X wins!


# **Algoritma Minimax Tic-tac-toe**

In [ ]:
import random
def draw_board(board):
    """
    Fungsi untuk menggambar papan permainan
    """
    print(board[0] + "|" + board[1] + "|" + board[2])
    print("-+-+-")
    print(board[3] + "|" + board[4] + "|" + board[5])
    print("-+-+-")
    print(board[6] + "|" + board[7] + "|" + board[8])

def check_win(board, player):
    """
    Fungsi untuk memeriksa apakah seorang pemain menang
    """
    win_positions =[[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
    for positions in win_positions:
        if board[positions[0]] == player and board[positions[1]] == player and board[positions[2]] == player:
            return True
    return False
    
def get_valid_moves(board):
    """
    Fungsi untuk mendapatkan daftar langkah yang valid
    """
    valid_moves = []
    for i in range(len(board)):
        if board[i] == " ":
            valid_moves.append(i)
    return valid_moves

def minimax(board, player, depth=0):
    """
    Fungsi untuk mengevaluasi nilai setiap langkah menggunakan algoritma
    minimax
    """
    if check_win(board, "X"):
        return -10 + depth
    elif check_win(board, "O"):
        return 10 - depth
    elif " " not in board:
        return 0
    if player == "O":
        best_score = float("-inf")
        for move in get_valid_moves(board):
            new_board = board[:]
            new_board[move] = player
            score = minimax(new_board, "X", depth + 1)
            best_score = max(best_score, score)
        return best_score
    else:
        best_score = float("inf")
        for move in get_valid_moves(board):
            new_board = board[:]
            new_board[move] = player
            score = minimax(new_board, "O", depth + 1)
            best_score = min(best_score, score)
        return best_score

def get_best_move(board, player):
    """
    Fungsi untuk mendapatkan langkah terbaik menggunakan algoritma
    minimax
    """
    best_score = float("-inf") if player == "O" else float("inf")
    best_move = None
    for move in get_valid_moves(board):
        new_board = board[:]
        new_board[move] = player
        score = minimax(new_board, "X" if player == "O" else "O")
        if player == "O" and score > best_score:
            best_score = score
            best_move = move
        elif player == "X" and score < best_score:
            best_score = score
            best_move = move
    return best_move
    
def tic_tac_toe():
    """
    Fungsi untuk menjalankan permainan Tic Tac Toe
    """
    board = [" ", " ", " ", " ", " ", " ", " ", " ", " "]
    players = ["X", "O"]
    turn = random.choice([0, 1])
    while True:
        draw_board(board)
        print("Player " + players[turn] + "'s turn.")
        if players[turn] == "X":
            move = int(input("Enter move (0-8): "))
            if board[move] != " ":
                print("Invalid move. Try again.")
                continue
        else:
            move = get_best_move(board, "O")
        board[move] = players[turn]
        if check_win(board, players[turn]):
            draw_board(board)
            print("Player " + players[turn] + " wins!")
            break
        if " " not in board:
            draw_board(board)
            print("It's a tie!")
            break
        turn = (turn + 1) % 2

if __name__ == "__main__":
    tic_tac_toe()


 | | 
-+-+-
 | | 
-+-+-
 | | 
Player X's turn.
Enter move (0-8): 0
X| | 
-+-+-
 | | 
-+-+-
 | | 
Player O's turn.
X| | 
-+-+-
 |O| 
-+-+-
 | | 
Player X's turn.
Enter move (0-8): 3
X| | 
-+-+-
X|O| 
-+-+-
 | | 
Player O's turn.
X| | 
-+-+-
X|O| 
-+-+-
O| | 
Player X's turn.
Enter move (0-8): 2
X| |X
-+-+-
X|O| 
-+-+-
O| | 
Player O's turn.
X|O|X
-+-+-
X|O| 
-+-+-
O| | 
Player X's turn.
Enter move (0-8): 7
X|O|X
-+-+-
X|O| 
-+-+-
O|X| 
Player O's turn.
X|O|X
-+-+-
X|O|O
-+-+-
O|X| 
Player X's turn.
Enter move (0-8): 8
X|O|X
-+-+-
X|O|O
-+-+-
O|X|X
It's a tie!


# **Tugas 05-1**

In [ ]:
# Base Classes
# PREDICATE - ON, ONTABLE, CLEAR, HOLDING, ARMEMPTY
class PREDICATE:
    def __str__ (self):
        pass
    
    def __repr__ (self):
        pass
    
    def __eq__ (self, other):
        pass
    
    def __hash__ (self):
        pass
    
    def get_action(self, world_state):
        pass

class Operation:
    def __str__ (self):
        pass
    
    def __repr__ (self):
        pass
    
    def __eq__ (self, other):
        pass
    
    def precondition (self):
        pass
    
    def delete (self):
        pass
    
    def add (self):
        pass


In [ ]:
class ON(PREDICATE):
    
    def __init__ (self, X, Y):
        self.X = X
        self.Y = Y

    def __str__ (self):
        return "ON({X},{Y})".format(X=self.X, Y=self.Y)
    
    def __repr__ (self):
        return self. __str__ ()
    
    def __eq__ (self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
    
    def __hash__ (self):
        return hash(str(self))
    
    def get_action(self, world_state):
        return StackOp(self.X, self.Y)

class ONTABLE(PREDICATE):
    
    def __init__ (self, X):
        self.X = X
    
    def __str__ (self):
        return "ONTABLE({X})".format(x=self.X)
    
    def __repr__ (self):
        return self. __str__ ()
    
    def __eq__ (self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
    
    def __hash__ (self):
        return hash(str(self))
    
    def get_action(self, world_state):
        return PutdownOp(self.X)


In [ ]:
class CLEAR(PREDICATE):
    def __init__ (self, X):
        self.X = X
    
    def __str__ (self):
        return "CLEAR({X})".format(X=self.X)
        self.X = X
    
    def __repr__ (self):
        return self.__str__ ()
    
    def __eq__ (self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__

    def __hash__ (self):
        return hash(str(self))
    
    def get_action(self, world_state):
        for predicate in world_state:
            # If Block is on another block, unstack
            if isinstance(predicate, ON) and predicate.Y == self.X:
                return UnstackOp(predicate.X, predicate.Y)
        return None

class HOLDING(PREDICATE):
    def __init__ (self, X):
        self.X = X
    
    def __str__ (self):
        return "HOLDING({X})".format(X=self.X)
    
    def __repr__ (self):
        return self.__str__ ()
    
    def __eq__ (self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__

    def __hash__ (self):
        return hash(str(self))
    
    def get_action(self, world_state):
        X = self.X
        # If block is on table, pick up
        if ONTABLE(X) in world_state:
            return PickupOp(X)
      # If block is on another block, unstack
        else:
            for predicate in world_state:
                if isinstance(predicate, ON) and predicate.X == X:
                    return UnstackOp(X, predicate.Y)

In [ ]:
class ARMEMPTY(PREDICATE):
    def __init__ (self):
        pass
    
    def __str__ (self):
        return "ARMEMPTY"
    
    def __repr__ (self):
        return self._str_()
    
    def __eq__ (self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
    
    def __hash__ (self):
        return hash(str(self))
    
    def get_action(self, world_state=[]):
        for predicate in world_state:
            if isinstance(predicate, HOLDING):
                return PutdownOp(predicate.X)
        return None
class StackOp(Operation):
    
    def __init__ (self, X, Y):
        self.X = X
        self.Y = Y
    
    def __str__ (self):
        return "STACK({X},{Y})".format(X=self.X, Y=self.Y)
    
    def __repr__ (self):
        return self. __str__ ()
    
    def __eq__ (self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
    
    def precondition (self):
        return [CLEAR(self.Y), HOLDING(self.X)]

    def delete (self):
        return [CLEAR(self.Y), HOLDING(self.X)]
    
    def add (self):
        return [ARMEMPTY(), ON(self.X, self.Y)]


In [ ]:
class UnstackOp(Operation):

    def __init__ (self, X, Y):
        self.X = X
        self.Y = Y
  
    def __str__ (self):
        return "UNSTACK({X},{Y})".format(X=self.X, Y=self.Y)
    
    def __repr__ (self):
        return self.__str__()
    
    def __eq__ (self, other):
      return self.__dict__ == other.__dict__ and self.__class__ == other.__class__

    def precondition (self):
        return [ARMEMPTY(), ON(self.X, self.Y), CLEAR(self.X)]
    
    def delete (self):
        return [ARMEMPTY(), ON(self.X, self.Y)]
    
    def add (self):
        return [CLEAR(self.Y), HOLDING(self.X)]

class PickupOp(Operation):
    
    def __init__ (self, X):
        self.X = X

    def __str__ (self):
        return "PICKUP({X})".format(X=self.X)
    
    def __repr__ (self):
        return self.__str__()
    
    def __eq__ (self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
    
    def precondition(self):
        return [CLEAR(self.X), ONTABLE(self.X), ARMEMPTY()]
    
    def delete(self):
        return [ARMEMPTY(), ONTABLE(self.X)]
    
    def add(self):
        return [HOLDING(self.X)]

class PutdownOp(Operation):
    
    def __init__(self, X):
        self.X = X
    
    def __str__(self):
        return "PUTDOWN({X})".format(X=self.X)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.__dict__ == other.__dict__ and self.__class__ == other.__class__
    
    def precondition(self):
        return [HOLDING(self.X)]
    
    def delete(self):
        return [HOLDING(self.X)]
    
    def add(self):
        return [ARMEMPTY(), ONTABLE(self.X)]
    
def isPredicate(obj):
    predicates = [ON, ONTABLE, CLEAR, HOLDING, ARMEMPTY]
    for predicate in predicates:
        if isinstance(obj, predicate):
            return True
    return False

def isOperation(obj):
    operations = [StackOp, UnstackOp, PickupOp, PutdownOp]
    for operation in operations:
        if isinstance(obj, operation):
            return True
    return False

def arm_status(world_state):
    for predicate in world_state:
        if isinstance(predicate, HOLDING):
            return predicate
    return ARMEMPTY()

In [ ]:
class GoalStackPlanner:
    
    def __init__ (self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
    def get_steps(self):
    # Store Steps
        steps = []
    # Program Stack
        stack = []
    # World State/Knowledge Base
        world_state = self.initial_state.copy()
    # Initially push the goal_state as compound goal onto the stack
        stack.append(self.goal_state.copy())
    # Repeat until the stack is empty
        while len(stack) != 0:
    # Get the top of the stack
            stack_top = stack[-1]
    # If Stack Top is Compound Goal, push its unsatisfied goals onto stack
            if type(stack_top) is list:
                compound_goal = stack.pop()
                for goal in compound_goal:
                    if goal not in world_state:
                        stack.append(goal)
      # If Stack Top is an action
            elif isOperation(stack_top):
      # Peek the operation
                  operation = stack[-1]
                  all_preconditions_satisfied = True
      # Check if any precondition is unsatisfied and push it onto program stack
                  for predicate in operation.delete():
                      if predicate not in world_state:
                          all_preconditions_satisfied = False
                          stack.append(predicate)
        # If all preconditions are satisfied, pop operation from stack and execute it
                  if all_preconditions_satisfied:
                      stack.pop()
                      steps.append(operation)
                      for predicate in operation.delete():
                          world_state.remove(predicate)
                      for predicate in operation.add():
                          world_state.append(predicate)
      # If Stack Top is a single satisfied goal
            elif stack_top in world_state:
                stack.pop()
      # If Stack Top is a single unsatisfied goal
            else:
                unsatisfied_goal = stack.pop()
      # Replace Unsatisfied Goal with an action that can complete it
                action = unsatisfied_goal.get_action(world_state)
                stack.append(action)
              # Push Precondition on the stack
                for predicate in action.precondition():
                    if predicate not in world_state:
                        stack.append(predicate)
        return steps

In [ ]:
if __name__ == '__main__':
    initial_state = [
        ON('2', '3'), ON('5', '6'), ON('4', '5'), 
        ONTABLE('1'),ONTABLE('2'), ONTABLE('3'),ONTABLE('4'),ONTABLE('5'), ONTABLE('6'),
        CLEAR('1'), CLEAR('2'), CLEAR('4'),
        ARMEMPTY()
    ]
    
    goal_state = [
        ON('6', '3'), ON('3', '1'), ON('2', '5'), ON('5', '4'),
        ONTABLE('1'), ONTABLE('4'),  
        CLEAR('6'), CLEAR('2'),
        ARMEMPTY()
    ]
    goal_stack = GoalStackPlanner(initial_state=initial_state,
goal_state=goal_state)
    steps = goal_stack.get_steps()
    print(steps)

[UNSTACK(4,5), PUTDOWN(4), UNSTACK(5,6), STACK(5,4), PICKUP(2), STACK(2,5), UNSTACK(2,3), PUTDOWN(2), PICKUP(3), STACK(3,1), PICKUP(6), STACK(6,3)]
